In [38]:
from ultralytics import YOLO
import time
import os
import pandas as pd
import cv2

In [ ]:
# read from tflite
model32 = YOLO('runs\\detect\\train\\weights\\best_saved_model\\best_float32.tflite')
model16 = YOLO('runs\\detect\\train\\weights\\best_saved_model\\best_float32.tflite')
modelpt = YOLO('runs\\detect\\train\\weights\\best.pt')
model_onnx = YOLO('runs\\detect\\train\\weights\\best.onnx')

## Inference time on image

In [40]:
def inference_image(model, image_path):
    # Measure inference time
    start_time = time.time()
    results = model(image_path)  # Run inference
    end_time = time.time()

    # Calculate FPS (1 / time per inference)
    inference_time = end_time - start_time
    fps = 1 / inference_time
    
    return inference_time, fps


# only pt can take list of images
def batch_inference(model, images):
    # Measure inference time
    start_time = time.time()
    results = model(images)  # Run inference
    end_time = time.time()

    # Calculate FPS (1 / time per inference)
    inference_time = end_time - start_time
    fps = 1 / inference_time
    
    return inference_time, fps

In [36]:
# Create an empty list to store results
results = []

# Iterate over test images
for img_path in os.listdir("datasets/images/test")[:20]:
    full_img_path = os.path.join("datasets/images/test", img_path)

    # Inference with model32
    inf_time, fps = inference_image(model32, full_img_path)
    results.append({
        "image": img_path,
        "model": "model32",
        "inference_time": inf_time,
        "fps": fps
    })

    # Inference with model16
    inf_time, fps = inference_image(model16, full_img_path)
    results.append({
        "image": img_path,
        "model": "model16",
        "inference_time": inf_time,
        "fps": fps
    })

    # Inference with modelpt
    inf_time, fps = inference_image(modelpt, full_img_path)
    results.append({
        "image": img_path,
        "model": "modelpt",
        "inference_time": inf_time,
        "fps": fps
    })

    # Inference with model_onnx
    inf_time, fps = inference_image(model_onnx, full_img_path)
    results.append({
        "image": img_path,
        "model": "model_onnx",
        "inference_time": inf_time,
        "fps": fps
    })

# Convert the results list to a pandas DataFrame
df = pd.DataFrame(results)

# Calculate the average inference time and FPS for each model
average_stats = df.groupby('model').agg(
    average_inference_time=('inference_time', 'mean'),
    average_fps=('fps', 'mean')
).reset_index()

# Print the DataFrame with the average results
print(average_stats)


image 1/1 c:\Users\BenJordan\Documents\Git\object-detection-YOLOv11\datasets\images\test\10007.jpg: 640x640 1 Parcel, 104.2ms
Speed: 12.0ms preprocess, 104.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Inference Time: 0.1186 seconds
FPS: 8.43

image 1/1 c:\Users\BenJordan\Documents\Git\object-detection-YOLOv11\datasets\images\test\10007.jpg: 640x640 1 Parcel, 152.1ms
Speed: 0.0ms preprocess, 152.1ms inference, 3.6ms postprocess per image at shape (1, 3, 640, 640)
Inference Time: 0.1569 seconds
FPS: 6.37

image 1/1 c:\Users\BenJordan\Documents\Git\object-detection-YOLOv11\datasets\images\test\10007.jpg: 416x640 1 Parcel, 68.1ms
Speed: 0.0ms preprocess, 68.1ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 640)
Inference Time: 0.0717 seconds
FPS: 13.95

image 1/1 c:\Users\BenJordan\Documents\Git\object-detection-YOLOv11\datasets\images\test\10007.jpg: 640x640 1 Parcel, 125.4ms
Speed: 0.0ms preprocess, 125.4ms inference, 0.0ms postprocess per image at 

## Webcam test

In [ ]:
# run on webcam
model32(0)

In [ ]:
import cv2
import time
import pandas as pd
from ultralytics import YOLO

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not access webcam")
    exit()

frame_count = 0
start_time = time.time()

results_list = []
end_time = start_time + 10

while time.time() < end_time:
    ret, frame = cap.read()
    if not ret:
        break
    frame_start_time = time.time()
    results = model32(frame)
    frame_end_time = time.time()
    inference_time = frame_end_time - frame_start_time
    fps = frame_count / (frame_end_time - start_time) if frame_count > 0 else 0
    results_list.append({"Frame": frame_count, "Inference Time (s)": inference_time, "FPS": fps})
    frame_count += 1
    cv2.imshow("Webcam", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

total_time = time.time() - start_time
fps = frame_count / total_time

df = pd.DataFrame(results_list)

cap.release()
cv2.destroyAllWindows()


0: 640x640 1 Person, 100.5ms
Speed: 6.0ms preprocess, 100.5ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Person, 98.5ms
Speed: 7.3ms preprocess, 98.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Person, 99.9ms
Speed: 4.8ms preprocess, 99.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Person, 109.3ms
Speed: 4.6ms preprocess, 109.3ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Person, 111.2ms
Speed: 3.4ms preprocess, 111.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Person, 118.8ms
Speed: 2.4ms preprocess, 118.8ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Person, 129.0ms
Speed: 3.8ms preprocess, 129.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Person, 109.7ms
Speed: 2.4ms preprocess, 109.7ms inference, 0.5ms postprocess per image at sha

In [66]:
df.head()

,Frame,Inference Time (s),FPS
0,0,0.104551,0.000000
1,1,0.126359,3.806508
2,2,0.111743,5.278639
3,3,0.114339,5.862385
4,4,0.143279,6.075138


In [69]:
print(df["Inference Time (s)"].mean())
print(df["FPS"].mean())

0.14124109815148747
6.472459767799961


In [ ]:
# convert to quantized tflite
model = YOLO('runs\\detect\\train\\weights\\best.pt')
model.export(format = 'tflite', int8=True, data = 'model/data.yaml')

Ultralytics 8.3.70  Python-3.11.0 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i9-13900H)
YOLO11n summary (fused): 238 layers, 2,583,127 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from 'runs\detect\train\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 9, 8400) (5.2 MB)

TensorFlow SavedModel: starting export with tensorflow 2.18.0...

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.48...
ONNX: export success  1.3s, saved as 'runs\detect\train\weights\best.onnx' (10.2 MB)
TensorFlow SavedModel: collecting INT8 calibration images from 'data=model/data.yaml'


Scanning C:\Users\BenJordan\Documents\Git\object-detection-YOLOv11\datasets\labels\val.cache... 5230 images, 31 backgrounds, 6 corrupt: 100%|██████████| 5230/5230 [00:00<?, ?it/s]

val: WARNING  C:\Users\BenJordan\Documents\Git\object-detection-YOLOv11\datasets\images\val\1057.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.1104478]
val: WARNING  C:\Users\BenJordan\Documents\Git\object-detection-YOLOv11\datasets\images\val\3581.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0338461]
val: WARNING  C:\Users\BenJordan\Documents\Git\object-detection-YOLOv11\datasets\images\val\4598.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.1328671]
val: WARNING  C:\Users\BenJordan\Documents\Git\object-detection-YOLOv11\datasets\images\val\7408.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.1384616]
val: WARNING  C:\Users\BenJordan\Documents\Git\object-detection-YOLOv11\datasets\images\val\7432.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.9266862 1.0029325]
val: WARNING  C:\Users\BenJordan\Documents\Git\obj